### Project aims to scrape Malabar Café in  Grand Hyatt Kochi, Bolgatty, Kochi

### Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver

### Loading the website URL

In [52]:
# creating a variable called 'url' and using the requests.get function to store content of the site to the variable data
url = 'https://www.hyattrestaurants.com/en/dining/india/kochi/malabar-caf%C3%A9-restaurant-menu#5f322c618741d25d205fdf52'
data = requests.get(url)

In [6]:
#creating a variable 'driver' and storing the chromedriver path to this variable
driver = webdriver.Chrome(executable_path=r"C:\Users\user\Desktop\Python\Projects\Web Scrapping\chromedriver")

C:\Users\user\AppData\Local\Temp\ipykernel_6128\906057503.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\user\Desktop\Python\Projects\Web Scrapping\chromedriver")


In [53]:
#Automating creation of a new window with selenium's driver
#Scrolling to the end of the page to ensure all information is captured
#rate-limiting requests after 2 secs
driver.get(url)
driver.execute_script("window.scrollTo(1,20000)")
time.sleep(2)

In [54]:
#Storing the content of the page to variable 'html'
html = driver.page_source

In [55]:
#writing content of the page to a new file to store data, this way we don't have to scrape Hyatt Malabar cafe everytime
with open("hyatt_malabar_cafe_menu.html", "w+",encoding='utf-8', errors='backslashreplace') as f:
    f.write(html)

In [57]:
#reading the content of Hyatt Malabar cafe and storing it to variable 'hyatt_full_data'
with open("hyatt_malabar_cafe_menu.html", "r",encoding='utf-8', errors='backslashreplace') as f:
    hyatt_full_data = f.read()

In [58]:
#here i'm using Beautiful soup's html parser to parse data into a more structured format and storing this in variable 'soup'
soup = BeautifulSoup(hyatt_full_data, 'html.parser')

### Manually do the inspection of Hyatt Malabar Cafe website to study the structure of the website to scrape data

### After inspection, I came to know that all the menu information is contained within a div with a class called 'listing_content'

In [59]:
hyatt_full_clean = soup.find_all("div", class_="listing_content")

In [60]:
# convert the content of 'menu-item-container' to a list so I can iterate easily
full_lunch_dinner_menu = list(hyatt_full_clean)

In [61]:
# To find the name of the item
full_lunch_dinner_menu[9].contents[3].text.strip()

'Kerala Beef Fry'

In [62]:
full_lunch_dinner_menu[9].contents[1].text.strip()
#Gives Index

'10'

In [63]:
# To find the description of the dishes
full_lunch_dinner_menu[9].contents[7].text.strip()


'Fried Spicy Beef. 183 Cal. Per 100g Serving'

In [64]:
# To find the price of the dishes
full_lunch_dinner_menu[9].contents[9].text.strip()

'₹ 700'

In [48]:
# Creating empty list to store data
item = []
item_description = []
item_price = []
# Only 69 items are present
for i in range(1,69):
    # Listing all items under 'listing_content'
    menu = list(soup.find_all("div", class_="listing_content"))[i]
    # Storing the values of name, price and description of each item as the loop goes through each iteration
    try:
        food = menu.contents[3].text.strip()
        food_description = menu.contents[7].text.strip()
        food_price = menu.contents[9].text.strip()
    except:
        food = ""
        food_description = ""
        food_price = ""
    # Append the list created initially with the respective values of name, description and price as the loop progresses
    item.append(food)
    item_description.append(food_description)
    item_price.append(food_price)

In [65]:
full_lunch_dinner_menu = pd.DataFrame({
    'Name':item,
    'Description':item_description,
    'Price':item_price
})

In [66]:
full_lunch_dinner_menu

,Name,Description,Price
0,"Pickled Cucumber, Cherry Tomato And Burrata Salad",Toasted Almonds. 176 Cal. Per 100g Serving,₹ 650
1,Caesar Salad,Chicken. 163 Cal. Per 100g Serving,₹ 600
2,Caesar Salad,"Prawn - Crispy Romaine, Chargrilled Chicken/ P...",₹ 750
3,Malabar Shrimp Cocktail,"Poached Shrimp With Black Pepper Mayonnaise, Q...",₹ 700
4,Malabar Café Cutlet,"Beetroot, Kandhari Chili, Fennel. 210 Cal. Per...",₹ 450
...,...,...,...
63,Dal Tadka,"Yellow Lentils, Garlic, Cumin, Red Chili. 172 ...",₹ 550
64,Parippu Pradhaman,"Sweet Lentils Simmered In Coconut Milk, Cane S...",₹ 250
65,Palada,"Rice ﬂakes Cooked In Milk, Sugar And Ghee. 207...",₹ 250
66,Gulab Jamun,Indian Fried Dumpling Soaked In Sugar Syrup. 1...,₹ 350


### Saving the dataFrame to a CSV file called "full_lunch_dinner_menu.csv" with the index column removed.

In [90]:
df = pd.DataFrame(full_lunch_dinner_menu)

# save the dataframe into csv file
df.to_csv('full_lunch_dinner_menu.csv', index=False)